In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from config import key
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb() #this is needed for SQL Alchemy because they need a MySQL Library 

### Store CSV into DataFrame and clean data 

The databases/df below were divided mantaining same order from original source. To review more detailg please go to https://www.who.int/healthinfo/statistics/mortality_rawdata/en/)


# mortality7

In [11]:
csv_file = "../Resources/Kaggle/master.csv"
df = pd.read_csv(csv_file)
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [12]:
df.sum(axis = 0, skipna = True)

country               AlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlbaniaAlba...
year                                                           55675008
sex                   malemalefemalemalemalefemalefemalefemalemalefe...
age                   15-24 years35-54 years15-24 years75+ years25-3...
suicides_no                                                     6748420
population                                                  51322158436
suicides/100k pop                                                356544
country-year          Albania1987Albania1987Albania1987Albania1987Al...
HDI for year                                                    6495.49
 gdp_for_year ($)     2,156,624,9002,156,624,9002,156,624,9002,156,6...
gdp_per_capita ($)                                            469225040
generation            Generation XSilentGeneration XG.I. GenerationB...
dtype: object

In [13]:
df.dtypes

country                object
year                    int64
sex                    object
age                    object
suicides_no             int64
population              int64
suicides/100k pop     float64
country-year           object
HDI for year          float64
 gdp_for_year ($)      object
gdp_per_capita ($)      int64
generation             object
dtype: object

In [28]:
suicide = df.iloc[:,:6]
suicide.head()

,country,year,sex,age,suicides_no,population
0,Albania,1987,male,15-24 years,21,312900
1,Albania,1987,male,35-54 years,16,308000
2,Albania,1987,female,15-24 years,14,289700
3,Albania,1987,male,75+ years,1,21800
4,Albania,1987,male,25-34 years,9,274300


In [17]:
# Connecting to SQL. To run this, remember to add a config file with your SQL key
rds_connection_string = (f"root:{key}@127.0.0.1/mortality")
engine = create_engine(f'mysql://{rds_connection_string}')

In [18]:
# Insert tables to SQL  
engine.table_names()
suicide.to_sql(name='suicide', con=engine, if_exists='append', index=True)

In [32]:
suicide_df = pd.read_sql_query('select * from suicide', con=engine)
suicide_df.head()

,index,country,year,age,suicides_no,population
0,0,Albania,1987,15-24 years,21,312900
1,1,Albania,1987,35-54 years,16,308000
2,2,Albania,1987,15-24 years,14,289700
3,3,Albania,1987,75+ years,1,21800
4,4,Albania,1987,25-34 years,9,274300


In [34]:
suicide_df.head()

,index,country,year,age,suicides_no,population
0,0,Albania,1987,15-24 years,21,312900
1,1,Albania,1987,35-54 years,16,308000
2,2,Albania,1987,15-24 years,14,289700
3,3,Albania,1987,75+ years,1,21800
4,4,Albania,1987,25-34 years,9,274300


In [37]:
suicide_df.drop(['index'], axis=1, inplace=True)

In [46]:
group = suicide_df.groupby(["year","country"]).sum()
group.head()

suicides_no  population
year country                                     
1985 Antigua and Barbuda            0       62574
     Argentina                   1988    27090800
     Australia                   1861    14562500
     Austria                     2091     7110115
     Bahamas                        1      203700

In [50]:
group["Suicide Rate per 100k"] = ((group["suicides_no"]/group["population"])*100000)  

In [53]:
group.tail(100)

suicides_no  population  \
year country                                                 
2014 Romania                              2209    18952235   
     Russian Federation                  26541   134713962   
     Saint Lucia                            14      165596   
     Saint Vincent and Grenadines            2      100676   
     Serbia                               1134     6801810   
     Seychelles                              4       83845   
     Singapore                             355     3689370   
     Slovakia                              554     5129686   
     Slovenia                              388     1951786   
     South Africa                          580    48912129   
     Spain                                3911    44172771   
     Suriname                              128      497701   
     Sweden                               1148     9114524   
     Switzerland                          1032     7809621   
     Thailand                             3952    61139919   
     Turkey                               1617    70931420   
     Turkmenistan                          136     4811227   
     Ukraine                              7968    40464165   
     United Kingdom                       4788    60570482   
     United States                       42769   297749735   
     Uruguay                               591     3177645   
     Uzbekistan                           2095    27313507   
2015 Antigua and Barbuda                     1       91889   
     Argentina                            3073    39699624   
     Armenia                                74     2795335   
     Australia                            3027    22240785   
     Austria                              1251     8219386   
     Belgium                              1867    10624356   
     Belize                                 26      319835   
     Brazil                              11163   191087085   
...                                        ...         ...   
     Seychelles                              7       85949   
     Singapore                             329     3719115   
     Slovenia                              422     1954632   
     South Africa                          482    49627460   
     Spain                                3604    44186528   
     Sweden                               1182     9114524   
     Switzerland                          1073     7892502   
     Thailand                             4205    61261276   
     Turkey                               1532    71880464   
     Turkmenistan                          133     4886514   
     Ukraine                              7574    40345446   
     United Kingdom                       4910    61082942   
     United States                       44189   300078511   
     Uruguay                               630     3190795   
2016 Armenia                                67     2400033   
     Austria                              1201     7484629   
     Croatia                               683     3567043   
     Cyprus                                 36      711344   
     Czech Republic                       1318     8929942   
     Grenada                                 0       79012   
     Hungary                              1761     8390370   
     Iceland                                40      268773   
     Lithuania                             822     2445296   
     Mauritius                              98      992432   
     Mongolia                              423     2167589   
     Netherlands                          1886    14239554   
     Qatar                                  68     2213121   
     Romania                              1953    16644905   
     Sweden                               1130     8125852   
     Thailand                             4117    53442001   

                                   Suicide Rate per 100k  
year country                                              
2014 Roman